In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\medici\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train = pd.read_csv('dataset/train.csv')

In [4]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


- 타겟 먼저 확인해 볼게요. 3번이 없는것을 확인했고 1,2,4,5의 개수를 파악했습니다. 
- 긍정은 4보다 5가 많았고 부정은 1보다 2가 많은것을 볼 수 있는데요. 과연 전체를 다 맞추는 작업과 부정과 긍정을 구분짓고 상대적으로 많은 2번, 5번으로 몰아 넣는다면 좀더 정확도 높지 않을까? 하는 생각을 해봤습니다. 

In [5]:
print((train.target==1).sum())
print((train.target==2).sum())
print((train.target==3).sum())
print((train.target==4).sum())
print((train.target==5).sum())

4500
8000
0
2500
10000


- 단순하게 4점은 5점으로 1점은 2점으로 변경해주었습니다.

train['target'] = train['target'].replace(4,5).replace(1,2)
train['target'].unique()

- train/val data set 분리

In [6]:
train_data, val_data = train_test_split(train, test_size=0.25, 
                                        stratify = train.target,
                                        random_state =0) # 25프로로 설정

In [7]:
# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)

In [8]:
print(train_data.shape)
print(val_data.shape)

(18750, 3)
(6250, 3)


In [9]:
# training 데이터에서 변수 추출
train_X = train_data.reviews
train_y = train_data.target 

# validation 데이터에서 변수 추출
val_X = val_data.reviews
val_y = val_data.target 

- nltk word_tokenize로 문장을 구분

In [10]:
nltk.word_tokenize(train_X[0])

['상품이', '저가', '원하는대로', '왔어요', '.']

- 특수문자 제거

In [11]:
delet = [] 
for i in range(len(train_X)):
    delet.append(re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·  ]',' ',train_X[i]))

In [12]:
train_token = []
for i in range(len(delet)):
    train_token.append(nltk.word_tokenize(delet[i]))


In [13]:
train_token[:3]

[['상품이', '저가', '원하는대로', '왔어요'],
 ['몇번',
  '세탁하고',
  '사용하니',
  '보풀이',
  '조금',
  '일어나요',
  '그래도',
  '금액도',
  '적당하고',
  '생각보다',
  '커서',
  '사용하기',
  '편해요'],
 ['너무',
  '크고',
  '무거워',
  '이동성은',
  '떨어져요',
  '밀리진',
  '않지만',
  '들고다니려니',
  '손목이',
  '아프네요',
  '4살',
  '딸',
  '머리',
  '감길때',
  '손목아파',
  '샀는데',
  '편하진',
  '않네요',
  '뒷',
  '머리',
  '감기기도',
  '불편하구요']]

## Word2Vec 학습
- train_X의 토큰인 train_token으로 학습합니다.

In [14]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 

In [15]:
model = Word2Vec(sentences = train_token, # train_X 토큰 
                     vector_size=100,      # 임베딩된 벡터 차원 
                     window=10,             # 컨텍스트 윈도우 크기
                     min_count=5,          # 5번 이하는 X 
                     workers=40,           # 학습 프로세스 수 
                     sg=1)                 # 0= CBOW, 1= Skip-Gram
                                           # skipgram = 중심단어로 주변단어 예측 

In [16]:
model.wv.most_similar('좋아요')

[('편하고', 0.9822671413421631),
 ('좋네요', 0.9820128083229065),
 ('배송빠르고', 0.9775317907333374),
 ('좋고', 0.9766674041748047),
 ('향도', 0.9750815033912659),
 ('좋습니다', 0.9729708433151245),
 ('싸고', 0.9723783135414124),
 ('저렴하고', 0.9709473252296448),
 ('깔끔하고', 0.9701974391937256),
 ('적당하고', 0.9699755311012268)]

In [17]:
model.wv.most_similar('안좋아요')

[('별로라', 0.9964783787727356),
 ('얇고', 0.9963861107826233),
 ('작음', 0.9961965680122375),
 ('ㅠㅠㅠ', 0.996020495891571),
 ('자국이', 0.9959438443183899),
 ('반품비가', 0.9959155321121216),
 ('아닌데', 0.9958221316337585),
 ('뻣뻣해서', 0.9957759380340576),
 ('묻어서', 0.9957319498062134),
 ('약하네요', 0.9956998229026794)]

- 피쳐 벡터 구하는 함수

In [18]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index_to_key)
    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[w])
    return feature_vector

In [19]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs


- get_dataset함수로 피쳐벡터를 받아와 logistic에 x값으로

In [20]:
x_train_w = get_dataset(train_token,model,100)

In [21]:
lr = LogisticRegression(max_iter=10000)

In [22]:
lr.fit(x_train_w,train_y)

LogisticRegression(max_iter=10000)

- Validation 데이터도 마찬가지로 작업해줘야 효과 있겠죠?

In [23]:
delet_v = [] 
for i in range(len(val_X)):
    delet_v.append(re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·  ]',' ',val_X[i]))

In [24]:
val_token = []
for i in range(len(delet_v)):
    val_token.append(nltk.word_tokenize(delet_v[i]))


In [25]:
x_val_w = get_dataset(val_token,model,100)

In [26]:
(lr.predict(x_val_w) == val_y).mean()

0.59184

In [27]:
train_all = []
for i in range(len(train['reviews'])):
    train_all.append(re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·  ]',' ',train['reviews'][i]))

In [28]:
all_token = []
for i in range(len(train_all)):
    all_token.append(nltk.word_tokenize(train_all[i]))


In [29]:
len(all_token)

25000

In [30]:
test = pd.read_csv('dataset/test.csv')

In [31]:
test['reviews']

0                                           채소가 약간 시들어 있어요
1        발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2                                 부들부들 좋네요 입어보고 시원하면 또 살게요
3            이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4             검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요
                               ...                        
24995                                         사용해보니 좋아요~^^
24996                   저렴한가격에. 질좋고. 핏좋고. 너무. 이쁘게. 입고다녀요..
24997    세트상품이라고 써있어서 그런줄 알고 구매했더니 단품이었네요 낚인 느낌도 들고 그러네...
24998                                       역시 로네펠트!! 좋아요.
24999    데싱 디바 써보고 갠찮아서 비슷 한줄 알앗더니 완전 별로예요ㅡㅡ3000언 더주고 디...
Name: reviews, Length: 25000, dtype: object

In [32]:
test_all = []
for i in range(len(test['reviews'])):
    test_all.append(re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·  ]',' ',test['reviews'][i]))

In [33]:
test_token = []
for i in range(len(test_all)):
    test_token.append(nltk.word_tokenize(test_all[i]))

In [34]:
test_w = get_dataset(test_token,model,100)

In [35]:
pred = lr.predict(test_w)

In [51]:
pred[:100]

array([5, 2, 5, 2, 2, 2, 2, 2, 5, 5, 2, 5, 5, 2, 2, 5, 2, 2, 5, 2, 5, 5,
       2, 5, 5, 5, 2, 5, 5, 2, 2, 2, 5, 2, 2, 2, 2, 5, 1, 2, 2, 5, 5, 2,
       2, 5, 5, 5, 5, 5, 5, 2, 5, 2, 2, 2, 5, 2, 5, 5, 5, 2, 2, 2, 2, 2,
       5, 2, 2, 5, 5, 2, 5, 5, 5, 2, 5, 2, 5, 2, 5, 2, 5, 5, 2, 5, 2, 2,
       2, 2, 2, 2, 5, 2, 1, 2, 1, 5, 5, 2], dtype=int64)

In [37]:
submission = pd.read_csv('dataset/sample_submission.csv')

In [38]:
submission['target'] = pred 

submission.to_csv("submission.csv",index=False)